---
# **INTRODUCCIÓN** 
---


Esta sección resume el análisis exploratorio y las decisiones tomadas en la fase inicial de preprocesamiento, necesarias para transformar los datos brutos en un formato óptimo para modelos de aprendizaje automático como Support Vector Machines (SVM). Se detallan las estrategias aplicadas y su justificación técnica.



## 1. Análisis de datos originales (archivo 01)

- Variables clave identificadas:
  - 15 variables categóricas de alta cardinalidad  
    *Ejemplo destacado:* ESTU_PRGM_ACADEMICO contiene 948 categorías distintas, planteando desafios para codificacion tradicional.
  - 6 variables numéricas
    Se incluyen variables con alta correlación  
    *Ejemplo:* coef_2 y coef_4 tienen una correlación de 0.82, surge la posible necesidad de combinar o eliminar variables correlacionadas.
  - Valores faltantes:
    Entre 4.3% y 6.3% en variables categóricas estrategicas

## 2. Decisiones para preprocesamiento

- Ingeniería de características:
  - Conversión de PERIODO a ANIO con el objetivo de capturar información
  - Cálculo de ratios, sumas y diferencias entre coeficientes 
  - Simplificación de niveles educativos en una escala numérica
  
- Tratamiento de valores faltantes:
  - Categóricas: imputación con la moda 
  - Numéricas: imputación con la mediana

- Codificación de variables categóricas:
  - Uso de Target Encoding para evitar la alta dimensionalidad que produciría One-Hot Encoding 

- Escalado de variables numéricas:
  - Uso de StandardScaler para transformar variables a una distribución estándar (media 0, desviación estándar 1) 

## 3. Estructura del pipeline

- Uso de ColumnTransformer para aplicar distintas transformaciones según el tipo de variable
- Integración de TargetEncoder
- El preprocesador se guarda en disco (preprocessor.joblib) para garantizar consistencia en producción

## 4. Resultados a lograr

- Dataset final con mas variables
- Valores faltantes reducidos a 0%
- Todas las variables categóricas convertidas a representaciones numéricas continuas
- Variables numéricas estandarizadas

## 5. Próximos pasos (archivo 03 - SVM)

- Entrenamiento de un modelo SVM requerirá:
  - Ajuste del parámetro C (control del margen de clasificación)
  - Uso de kernel RBF para capturar relaciones no lineales
  - Posible uso de pesos por clase para corregir desbalance leve

- Consideraciones adicionales:
  - Aplicar validación cruzada estratificada
  - Posible reducción de dimensionalidad post-preprocesamiento (por ejemplo, con PCA o selección de características)



---
# **PREPROCESAMIENTO**
---

Acontinuacion se realizara el preprocesamiento completo de los datos de entrenamiento y de prueba para el modelo de machine learning:
- Limpieza
- Ingeniería de características
- Imputación de valores faltantes
- Codificación de variables categóricas
- Escalamiento de variables numéricas
- Exportación de los datos transformados

## 1. Importación de librerías
Se importan módulos para:
- Manipulación de datos: pandas, numpy
- Preprocesamiento y pipelines: sklearn
- Codificación categórica: TargetEncoder
- Serialización: joblib

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler
import joblib


## 2. Eliminación de columnas duplicadas
Se eliminan duplicados de dos formas:
- eliminar_columnas_duplicadas_por_contenido: elimina columnas aunque tengan diferente nombre, si todos sus valores son idénticos.
- df.loc[:, ~df.columns.duplicated()]: elimina columnas con nombres duplicados, aunque sus contenidos difieran.


In [3]:
def eliminar_columnas_duplicadas_por_contenido(df):
    columnas_a_eliminar = []
    columnas = df.columns
    for i in range(len(columnas)):
        for j in range(i + 1, len(columnas)):
            col1 = columnas[i]
            col2 = columnas[j]
            # Verificar si tienen los mismos valores (ignorando NaNs)
            if df[col1].equals(df[col2]):
                columnas_a_eliminar.append(col2)
    return df.drop(columns=columnas_a_eliminar)

## 3. Ingeniería de características personalizada
La función feature_engineering(df) transforma el dataset original añadiendo nuevas variables y codificando otras.  
crea varias nuevas columnas a partir de las existentes y aplica el tratamiento de valores faltantes:

Transformaciones directas:
- ANIO: Calculada a partir de PERIODO.
- FAMI_EDUCACIONPADRE_SIMPL, FAMI_EDUCACIONMADRE_SIMPL: Simplificaciones de nivel educativo.

Interacciones entre coeficientes:
- COEF_RATIO: Relación entre coef_1 y coef_2.
- COEF_SUMA: Suma de coef_1 a coef_4.
- COEF_PROMEDIO: Promedio de los coeficientes.
- COEF_MAX: Máximo entre coef_1 a coef_4.
- COEF_MIN: Mínimo entre coef_1 a coef_4.
- COEF_RANGO: Diferencia entre el máximo y el mínimo.
- COEF_MEJORA: Cambio entre coef_4 y coef_1.

Mapeos categóricos:
- ESTU_HORAS_TRABAJO_NUM: Conversión de horas de trabajo a valores numéricos.
- COEF_X_TRABAJO: Multiplicación entre COEF_PROMEDIO y ESTU_HORAS_TRABAJO_NUM.
- ESTRATO_NUM: Conversión del estrato a número.
- VULNERABLE: Indicador binario de vulnerabilidad.
- ES_INGENIERIA: Identificación si el programa académico contiene "INGENIERÍA".
- ES_LICENCIATURA: Identificación si el programa académico contiene "LICENCIATURA".


In [ ]:
def feature_engineering(df):
    df = df.copy()
    # Eliminar columnas duplicadas
    df = eliminar_columnas_duplicadas_por_contenido(df)
    df = df.loc[:, ~df.columns.duplicated()]
    # Conversión de PERIODO a año
    df['ANIO'] = (df['PERIODO'] - 20183) // 100 + 2018
    # Interacciones entre coeficientes
    df['COEF_RATIO'] = df['coef_1'] / (df['coef_2'] + 1e-6)
    df['COEF_SUMA'] = df[['coef_1', 'coef_2', 'coef_3', 'coef_4']].sum(axis=1)
    # Simplificación educación padres
    educ_map = {'completa': 2, 'incompleta': 1, 'otros': 0}
    for col in ['FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE']:
        df[col + '_SIMPL'] = (
            df[col]
            .str.extract('(completa|incompleta)')[0]
            .fillna('otros')
            .map(educ_map))
    # basadas en coeficientes
    df['COEF_PROMEDIO'] = df[['coef_1', 'coef_2', 'coef_3', 'coef_4']].mean(axis=1)
    df['COEF_MAX'] = df[['coef_1', 'coef_2', 'coef_3', 'coef_4']].max(axis=1)
    df['COEF_MIN'] = df[['coef_1', 'coef_2', 'coef_3', 'coef_4']].min(axis=1)
    df['COEF_RANGO'] = df['COEF_MAX'] - df['COEF_MIN']
    df['COEF_MEJORA'] = df['coef_4'] - df['coef_1']
    # Horas de trabajo en formato numérico
    trabajo_map = {
        '0': 0,
        'Menos de 10 horas': 1,
        'Entre 11 y 20 horas': 2,
        'Entre 21 y 30 horas': 3,
        'Más de 30 horas': 4}
    df['ESTU_HORAS_TRABAJO_NUM'] = df['ESTU_HORASSEMANATRABAJA'].map(trabajo_map)
    df['COEF_X_TRABAJO'] = df['COEF_PROMEDIO'] * df['ESTU_HORAS_TRABAJO_NUM']
    # Estrato en formato numérico y vulnerabilidad
    estrato_map = {
        'Estrato 1': 1,
        'Estrato 2': 2,
        'Estrato 3': 3,
        'Estrato 4': 4,
        'Estrato 5': 5,
        'Estrato 6': 6,}
    df['ESTRATO_NUM'] = df['FAMI_ESTRATOVIVIENDA'].map(estrato_map)
    df['VULNERABLE'] = ((df['ESTRATO_NUM'] <= 2) & (df['FAMI_TIENELAVADORA'] == 'No')).astype(int)
    # Clasificación por programa académico
    df['ES_INGENIERIA'] = df['ESTU_PRGM_ACADEMICO'].str.contains('INGENIERIA', case=False, na=False).astype(int)
    df['ES_LICENCIATURA'] = df['ESTU_PRGM_ACADEMICO'].str.contains('LICENCIATURA', case=False, na=False).astype(int)
    # Tratamiento de valores faltantes
    # Categóricas: imputación con moda
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    for col in categorical_cols:
        mode = df[col].mode()
        if not mode.empty:
            df[col] = df[col].fillna(mode[0])
    # Numéricas: imputación con mediana
    numerical_cols = df.select_dtypes(include=['number']).columns
    for col in numerical_cols:
        df[col] = df[col].fillna(df[col].median())
    return df.drop(columns=['PERIODO'], axis=1)



## 4. Carga de datos y aplicacion de ingenieria de caracteristicas
- Lectura de train.csv y test.csv
- Aplicación de feature_engineering() a ambos

In [8]:

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# Aplicar ingeniería de características
df_train = feature_engineering(df_train)
df_test = feature_engineering(df_test)


## 5. Separación de variables predictoras y objetivo
- En train:
  - X_train: variables predictoras
  - y_train: variable objetivo (RENDIMIENTO_GLOBAL)
- En test: se elimina la columna ID, resultando en X_test


In [11]:

X_train = df_train.drop(columns=['ID', 'RENDIMIENTO_GLOBAL'])
y_train = df_train['RENDIMIENTO_GLOBAL']
X_test = df_test.drop(columns=['ID'], errors='ignore')

## 6. Definición de columnas numéricas y categóricas
Se identifican automáticamente:
- Numéricas: con select_dtypes(include=np.number)
- Categóricas: con select_dtypes(exclude=np.number)

In [12]:

numeric_features = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=np.number).columns.tolist()


## 7. Creación del pipeline de preprocesamiento
Se usa un ColumnTransformer con dos pipelines:

- Para variables categóricas:
  - Imputación: valor más frecuente (SimpleImputer)
  - Codificación: TargetEncoder

- Para variables numéricas:
  - Imputación: mediana
  - Escalamiento: StandardScaler

In [13]:
preprocessor = ColumnTransformer([
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', TargetEncoder())
    ]), categorical_features),
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), numeric_features)])


## 8. Aplicación del preprocesador

Ajustar con train, transformar test
- fit_transform en X_train con y_train
- transform en X_test

In [14]:

X_train_prep = preprocessor.fit_transform(X_train, y_train)
X_test_prep = preprocessor.transform(X_test)


## 9. Guardado de resultados
Se exportan:
- El pipeline entrenado: preprocessor.joblib
- Datos de entrenamiento: X_train_prep.csv, y_train.csv
- Datos de prueba: X_test_prep.csv

In [16]:
joblib.dump(preprocessor, 'preprocessor.joblib')
pd.DataFrame(X_train_prep).to_csv('X_train_prep.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
pd.DataFrame(X_test_prep).to_csv('X_test_prep.csv', index=False)

---
# **VALIDACIÓN**
---

## 1. Tamaño de los datos

- El conjunto de prueba (X_test) mantiene el mismo número de observaciones y columnas antes y después del preprocesamiento:
  - Dimensión original: (296786, 33)
  - Dimensión después del preprocesamiento: (296786, 33)
- Esto indica que el pipeline no eliminó ni duplicó columnas ni filas, garantizando integridad en la transformación.

In [17]:
# crearemos un DataFrame para la validacion
columnas_train = X_train.columns.tolist()
df_prep = pd.DataFrame(X_test_prep, columns=columnas_train)

# 1. Tamaño antes y después
print("1) Tamaño de los datos:")
print(f"   - X_test original:      {X_test.shape}")
print(f"   - X_test preprocesado:  {df_prep.shape}\n")


1) Tamaño de los datos:
   - X_test original:      (296786, 33)
   - X_test preprocesado:  (296786, 33)



## 2. Columnas (antes y después)

- Las columnas presentes antes y después del preprocesamiento son idénticas en nombre y cantidad (33 en total).
- Esto se debe a que el preprocessor está configurado para aplicar transformaciones in-place (como escalado y codificación), sin alterar el orden ni los nombres de las variables.

In [18]:
# 2. Listado de columnas antes y después
print("2) Columnas:")
print("   - Columnas originales:")
print(f"     {X_test.columns.tolist()}\n")
print("   - Columnas preprocesadas:")
print(f"     {columnas_train}\n")


2) Columnas:
   - Columnas originales:
     ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR', 'FAMI_EDUCACIONMADRE', 'coef_1', 'coef_2', 'coef_3', 'coef_4', 'ANIO', 'COEF_RATIO', 'COEF_SUMA', 'FAMI_EDUCACIONPADRE_SIMPL', 'FAMI_EDUCACIONMADRE_SIMPL', 'COEF_PROMEDIO', 'COEF_MAX', 'COEF_MIN', 'COEF_RANGO', 'COEF_MEJORA', 'ESTU_HORAS_TRABAJO_NUM', 'COEF_X_TRABAJO', 'ESTRATO_NUM', 'VULNERABLE', 'ES_INGENIERIA', 'ES_LICENCIATURA']

   - Columnas preprocesadas:
     ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_

## 3. Estadísticas descriptivas (cambio de escala)

- Variables como coef_1 a coef_4 y ANIO, que tenían valores originales entre 0 y 1 (o constantes), fueron transformadas a una nueva escala.
- Después del preprocesamiento, todas las variables numéricas presentan:
  - Valores centrados y escalados con StandardScaler
  - Media cercana a 1.49 en variables codificadas (por efecto del TargetEncoder)
  - Menor dispersión y rango de valores ajustado a una escala numérica continua

In [19]:
# 3. Estadísticas descriptivas (numéricas)
print("3) Estadísticas descriptivas (numéricas):")
print("   - Antes:")
print(X_test.select_dtypes(include=np.number).describe().T[['count','mean','std','min','max']].head(5))
print("\n   - Después:")
print(df_prep.select_dtypes(include=np.number).describe().T[['count','mean','std','min','max']].head(5))
print()


3) Estadísticas descriptivas (numéricas):
   - Antes:
           count         mean       std     min       max
coef_1  296786.0     0.268235  0.121352     0.0     0.663
coef_2  296786.0     0.259989  0.094596     0.0     0.484
coef_3  296786.0     0.262855  0.059635     0.0     0.322
coef_4  296786.0     0.262450  0.067587     0.0     0.331
ANIO    296786.0  2018.000000  0.000000  2018.0  2018.000

   - Después:
                                   count      mean       std       min  \
ESTU_PRGM_ACADEMICO             296786.0  1.492128  0.286602  0.087337   
ESTU_PRGM_DEPARTAMENTO          296786.0  1.491732  0.101436  1.237766   
ESTU_VALORMATRICULAUNIVERSIDAD  296786.0  1.491927  0.269067  0.760932   
ESTU_HORASSEMANATRABAJA         296786.0  1.491188  0.128043  1.248992   
FAMI_ESTRATOVIVIENDA            296786.0  1.491693  0.190050  0.791987   

                                     max  
ESTU_PRGM_ACADEMICO             2.083367  
ESTU_PRGM_DEPARTAMENTO          1.873358  
ESTU_VALO

## 4. Valores faltantes

- Antes del preprocesamiento:
  - Varias variables presentaban miles de valores faltantes (por ejemplo: ESTRATO_NUM, FAMI_EDUCACIONMADRE, FAMI_TIENEINTERNET, etc.)
- Después del preprocesamiento:
  - Todos los valores faltantes fueron imputados exitosamente  
  - Esto garantiza que el modelo no se verá afectado por datos incompletos en la fase de predicción

In [25]:
# 4. Conteo de valores faltantes
print("4) Valores faltantes:")
print(df_prep.isnull().sum())


4) Valores faltantes:
ESTU_PRGM_ACADEMICO               0
ESTU_PRGM_DEPARTAMENTO            0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
FAMI_TIENELAVADORA                0
FAMI_TIENEAUTOMOVIL               0
ESTU_PRIVADO_LIBERTAD             0
ESTU_PAGOMATRICULAPROPIO          0
FAMI_TIENECOMPUTADOR              0
FAMI_EDUCACIONMADRE               0
coef_1                            0
coef_2                            0
coef_3                            0
coef_4                            0
ANIO                              0
COEF_RATIO                        0
COEF_SUMA                         0
FAMI_EDUCACIONPADRE_SIMPL         0
FAMI_EDUCACIONMADRE_SIMPL         0
COEF_PROMEDIO                     0
COEF_MAX                          0
COEF_MIN                          0
COEF_RANGO                        0
COEF_MEJORA                       0
ESTU_H

## 5. Muestra de las primeras 5 filas

- Antes del preprocesamiento:
  - Las variables categóricas aparecen como texto (e.g., "ADMINISTRACION DE EMPRESAS", "Si", "Estrato 2")
  - Las variables numéricas tienen escalas originales
  
- Después del preprocesamiento:
  - Las variables categóricas han sido convertidas a valores numéricos continuos, mediante Target Encoding
  - Las variables numéricas han sido estandarizadas (media 0, desviación estándar 1)
  - El resultado es un dataset numérico puro, listo para ser usado por algoritmos como SVM


In [26]:
# 5. Muestra puntual (head) lado a lado
print("5) Muestra de las primeras 5 filas:")
print("   -- Original --")
print(X_test.head(5))
print("\n   -- Preprocesado --")
print(df_prep.head(5))
print()


5) Muestra de las primeras 5 filas:
   -- Original --
                      ESTU_PRGM_ACADEMICO ESTU_PRGM_DEPARTAMENTO  \
0                          TRABAJO SOCIAL                BOLIVAR   
1  ADMINISTRACION COMERCIAL Y DE MERCADEO              ANTIOQUIA   
2                  INGENIERIA MECATRONICA                 BOGOTÁ   
3                      CONTADURIA PUBLICA                  SUCRE   
4              ADMINISTRACION DE EMPRESAS              ATLANTICO   

             ESTU_VALORMATRICULAUNIVERSIDAD ESTU_HORASSEMANATRABAJA  \
0                          Menos de 500 mil       Menos de 10 horas   
1  Entre 2.5 millones y menos de 4 millones     Entre 21 y 30 horas   
2    Entre 1 millón y menos de 2.5 millones                       0   
3    Entre 1 millón y menos de 2.5 millones     Entre 21 y 30 horas   
4  Entre 2.5 millones y menos de 4 millones     Entre 11 y 20 horas   

  FAMI_ESTRATOVIVIENDA FAMI_TIENEINTERNET  \
0            Estrato 3                 Si   
1            Estrato

## 6. Nuevas features añadidas

- Durante la ingeniería de características se generaron 11 nuevas variables:
  - Relacionadas con coeficientes:
    - COEF_PROMEDIO, COEF_MAX, COEF_MIN, COEF_RANGO, COEF_MEJORA
  - Transformaciones numéricas de variables categóricas:
    - ESTU_HORAS_TRABAJO_NUM, ESTRATO_NUM
  - Variables de interacción o categorización:
    - COEF_X_TRABAJO, VULNERABLE, ES_INGENIERIA, ES_LICENCIATURA

Estas nuevas variables amplían la representación de la información y capturan relaciones relevantes que podrían mejorar el desempeño del modelo.

In [27]:
# 6. Resumen de nuevas features añadidas
nuevas = [
    'COEF_PROMEDIO', 'COEF_MAX', 'COEF_MIN', 'COEF_RANGO', 'COEF_MEJORA',
    'ESTU_HORAS_TRABAJO_NUM', 'COEF_X_TRABAJO',
    'ESTRATO_NUM', 'VULNERABLE',
    'ES_INGENIERIA', 'ES_LICENCIATURA'
]
print("6) Nuevas features añadidas:")
for f in nuevas:
    print(f"   - {f}")

6) Nuevas features añadidas:
   - COEF_PROMEDIO
   - COEF_MAX
   - COEF_MIN
   - COEF_RANGO
   - COEF_MEJORA
   - ESTU_HORAS_TRABAJO_NUM
   - COEF_X_TRABAJO
   - ESTRATO_NUM
   - VULNERABLE
   - ES_INGENIERIA
   - ES_LICENCIATURA
